# Creating chemical states

After defining and creating a [ChemicalSystem](https://reaktoro.org/api/classReaktoro_1_1ChemicalSystem.html) object, we are equipped to create chemical states. In applications, we often deal with either a *single chemical state* or *a collection of them* (as in reactive transport, for example) with fixed temperature, pressure, and species amounts specified for each state. [ChemicalState](https://reaktoro.org/api/classReaktoro_1_1ChemicalState.html) class is commonly use for:
* **specifying initial conditions** for chemical equilibrium and kinetics calculations
* **storing computed states** resulting from the above computations

First, we consider an example of the chemical system with a gaseous phase only and corresponding [ChemicalState](https://reaktoro.org/api/classReaktoro_1_1ChemicalState.html) instance:

In [ ]:
from reaktoro import *

db = NasaDatabase("nasa-cea")

gases = GaseousPhase("CO2 O2 H2 H2O CH4 CO")

system = ChemicalSystem(db, gases)

# Providing initial chemical state
state = ChemicalState(system)
state.temperature(1000, "celsius") # remove this line to see the default value
state.pressure(10, "MPa")          # remove this line to see the default value
state.set("CO2", 0.1, "mol")
state.set("O2" , 0.2, "mol")
state.set("H2" , 0.3, "mol")
state.set("H2O", 0.4, "mol")
state.set("CH4", 0.5, "mol")
state.set("CO" , 0.0, "mol")       # remove this line to see the default value

# Print the obtained chemical state
print("INITIAL STATE")
print(state)

> **Note**: The table above **does not** contain *thermodynamic properties of the system*; only temperature, pressure, species and element amounts, and the electric charge.


> **Note**: By default, a [ChemicalState](https://reaktoro.org/api/classReaktoro_1_1ChemicalState.html) object is initialized with the following conditions:
> * 273.15 K (or 25 °C)
> * 10<sup>5</sup> Pa (or 1 bar)
> * 10<sup>-16</sup> mol as the amounts of every species.
> Zero is a numerically problematic value for species amounts; that's why we set it to a very small positive value instead. This should not be an issue for you in most cases, but if for some reason 10<sup>-16</sup> is not small enough, you can use the method `ChemicalState.setSpeciesAmount(1e-40)` to set a common amount value for all species in the system.

**TASK**: 
1) consider a more complicated chemical system containing
* mineral phases of Calcite, Dolomite, Quartz, Halite
* aqueous phase generated automatically based on the elements contained in the mineral phases
and 
2) create a chemical state with the following initial composition of the species per 1 kg of water at 50 °C and 5 atm:

| Aqueous species | Value     |
|-----------------|-----------|
| Na<sup>+</sup>  | 0.10 mol  |
| Cl<sup>-</sup>  | 0.10 mol  |
| Mg<sup>2+</sup> | 0.50 mmol |
| Ca<sup>2+</sup> | 0.50 mmol |
| Calcite         | 1 mg      |
| Dolomite        | 1 ug      |
| Quartz          | 1 kmol    |


In [ ]:
# -------------------------------------- #
# Code cell for the task
# -------------------------------------- #

## Setting surface area

Providing surface areas between some phases is an **important functionality in modelling chemical kinetics** in which *heterogeneous reactions* (i.e., reactions containing species from different phases, such as mineral and gas dissolution reactions) are common:

In [ ]:
db = PhreeqcDatabase("pitzer.dat")

solution = AqueousPhase()
minerals = MineralPhases("Calcite Quartz") # HINT: adding Dolomite

system = ChemicalSystem(db, solution, minerals)

state = ChemicalState(system)
state.set("H2O"     , 1.00, "kg")

state.surfaceArea("AqueousPhase", "Calcite", 1.2, "m2")
state.surfaceArea("AqueousPhase", "Quartz" , 1.4, "m2")
# HINT: adding surface area between Dolomite and AqueousPhase

print(state)

Initially, [ChemicalState](https://reaktoro.org/api/classReaktoro_1_1ChemicalState.html) contains no surface areas. As you set surface areas for some phase pairs, interphase surface information is saved in the [ChemicalState](https://reaktoro.org/api/classReaktoro_1_1ChemicalState.html) object. To access the phase pairs for which surfaces areas have been set and retrieve their values can be done as follows:

In [ ]:
print("EXISTING SURFACES IN THE CHEMICAL STATE OBJECT")
for [i, j] in state.surfaces():
    name1 = system.phase(i).name()      # the name of the first phase in the phase pair
    name2 = system.phase(j).name()      # the name of the other phase in the phase pair
    surfarea = state.surfaceArea(i, j)  # the surface area of this phase pair
    print(f"Surface area between {name1} and {name2}: {surfarea} m2")